In [4]:
from bs4.builder import FAST
from networkx.algorithms.tree import minimum_spanning_tree
from pyarrow.dataset import dataset
from scipy.constants import fermi
from sqlalchemy.dialects.mssql.information_schema import columns

from util.data_loader import DataLoader

movielens = DataLoader().main()
user_movie_matrix = movielens.train.pivot(index="user_id", columns="movie_id", values="rating")

user_movie_matrix[user_movie_matrix<4]=0
user_movie_matrix[user_movie_matrix.isnull()]=0
user_movie_matrix[user_movie_matrix>=4]=1
user_movie_matrix.head()


C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

movie_id,1,2,3,4,5,6,7,8,9,10,...,62000,62113,62293,62344,62394,62801,62803,63113,63992,64716
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
from mlxtend.frequent_patterns import apriori
freq_movies = apriori(user_movie_matrix,min_support=0.1, use_colnames=True)
freq_movies = freq_movies.sort_values(by="support", ascending=False)
freq_movies.head()

,support,itemsets
42,0.415,(593)
23,0.378,(318)
21,0.368,(296)
19,0.362,(260)
25,0.320,(356)


In [14]:
from mlxtend.frequent_patterns import association_rules, apriori

rules = association_rules(freq_movies,min_threshold=2, metric="lift")
rules.sort_values(by="lift",ascending=False)[["antecedents","consequents","lift"]].head()


,antecedents,consequents,lift
203,(4993),(5952),5.459770
202,(5952),(4993),5.459770
582,"(1196, 1198)","(1291, 260)",4.669188
579,"(1291, 260)","(1196, 1198)",4.669188
578,"(1291, 1196)","(260, 1198)",4.171359


In [1]:
from util.models import RecommendResult, Dataset
from base_recommender import BaseRecommender
from collections import defaultdict, Counter
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
np.random.seed(0)

class AssociationRecommender(BaseRecommender):
    def recommend(self, dataset, **kwargs)->RecommendResult:

        min_support = kwargs.get("min_support",0.1)
        min_threshold = kwargs.get("min_threshold",1)
        user_movie_matrix = dataset.train.pivot(index = "user_id", columns="movie_id", values = "rating")
        user_movie_matrix[user_movie_matrix<4]=0
        user_movie_matrix[user_movie_matrix.isnull()]=0
        user_movie_matrix[user_movie_matrix>=4]=1

        popular_movies = apriori(user_movie_matrix, min_support=min_support, use_colnames=True, low_memory=True)
        popular_movies_link = association_rules(popular_movies, metric="lift", min_threshold=min_threshold)

        # pred_love_items = defaultdict(list)
        # user_evaluated_movies = dataset.train.groupby("user_id").agg({"movie_id":list})["movie_id"].to_dict()
        # movielens_train_high_rating = dataset.train[dataset.train.rating>=4]
        # for user_id,data in movielens_train_high_rating.groupby("user_id"):
        #     input_data = data.sort_values("timestamp",ascending=False)["movie_id"].tolist()[:5]
        #     matched_flags = popular_movies_link.antecedents.apply(lambda x:len(set(input_data)&x))>=1
        #     consequent_movies = []
        #     for i, row in popular_movies_link[matched_flags].sort_values("lift",ascending=False).iterrows():
        #         consequent_movies.extend(row["consequents"])
        #     counter = Counter(consequent_movies)
        #     for movie_id, movie_cnt in counter.most_common():
        #         if movie_id not in user_evaluated_movies[user_id]:
        #             pred_love_items[user_id].append(movie_id)
        #         if len(pred_love_items[user_id])==10:
        #             break
        # return RecommendResult(dataset.test.rating, pred_love_items)

    #유저가 좋아할듯한 영화 10개까지 추천
    #유저가 이미 평가한 영화들-> 평점이 4점이상인 영화들 high rate-> user_id, data for문으로 순회  highrate.groupby해서 -> 유저가 평가한 영화들중 최신 5개들 리스트로
    #그거를 link의 antecedent와 교집합해서 1개라도 존재하면 true가 되고 아니면 false가 되게끔 link를 ㅇㅇ matched_flags로 -> consequent_movies =[] ->
    #for문 돌면서 i, row popular_movies_link[matched_flags]에서 sort해서 가져오기하나씩 그후 row['consequents']를 extend해주기 consequent_movies에서
    #counter에 몇번 카운트 되엇는지 세기 ->카운터에서 가장많이 카운트 된애 하나씩 꺼내면서 그 영호 id가 유저가 안본거면 예측에 추가 그후 10개까지만
        #평점이 높은 영화들중 유저 id와 data추출 -> 유저가 최근에 본 5개와 관련된거 하나씩 꺼내옴 ->많이뽑힌거 순으로  유저가 안본 영화를 10개까지 추천
        pred_love_items = defaultdict(list)
        high_rate_movies = dataset.train[dataset.train.rating>=4]
        user_rated_movies = dataset.train.groupby("user_id").agg({"movie_id":list})["movie_id"].to_dict()
        for user_id, data in high_rate_movies.groupby("user_id"):
            latest_user_watch = data.sort_values(by="timestamp", ascending=False)["movie_id"][:5]
            match = popular_movies_link.antecedents.apply(lambda x:len(set(latest_user_watch)&x))>=1
            consequent_movies = []
            for i, row in popular_movies_link[match].iterrows():
                consequent_movies.extend(row['consequents'])
            counter = Counter(consequent_movies)
            for movie_id, cnt in counter.most_common():
                if movie_id not in user_rated_movies[user_id]:
                    pred_love_items[user_id].append(movie_id)
                if len(pred_love_items[user_id])==10:
                    break
        return RecommendResult(dataset.test.rating, pred_love_items)





if __name__ =="__main__":
    AssociationRecommender().eval()

C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv(os.path.join(self.data_path,"movies.dat"), encoding="latin-1", names=cols,
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tags = pd.read_csv(os.path.join(self.data_path, "tags.dat"), encoding='latin-1',
C:\Users\USER\Desktop\추천 시스템 입문\util\data_loader.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as re

rmse: 0.000, recall: 0.036, precision:0.011
